<a href="https://colab.research.google.com/github/Suvarnaph/hello-world/blob/master/ImageLoader_V6_ShoulderOnly.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
#Import statements
from google.colab import drive
drive.mount('/content/drive/')

from __future__ import absolute_import, division, print_function
from pathlib import Path
import re
import os
from os import getcwd
from os.path import exists, isdir, isfile, join
import shutil
import numpy as np
import pandas as pd
import glob
import cv2

%matplotlib inline
from matplotlib import pyplot as plt

Drive already mounted at /content/drive/; to attempt to forcibly remount, call drive.mount("/content/drive/", force_remount=True).


In [0]:
#Testing : Extracting only shoulder
class Image_Loader:
  
    def __init__(self, root_path, source_dir_name, size, img_read_format, img_type = ('*.jpg', '*.png')):
        self.project_path = root_path #root directory path of Mura-v1.1 dataset
        self.root = Path(self.project_path)
        self.source_directory = Path.joinpath(self.root, source_dir_name) #path from which image has to be read - Ex: train/test complete path
        Path.joinpath(self.root, source_dir_name)
        self.tmp = Path.joinpath(self.source_directory,"XR_SHOULDER")
        #print(self.tmp)
        #print(self.source_directory)
        for study_category in self.source_directory.iterdir():
          if(os.path.isdir(study_category)):#Only check for directories
            if study_category == (Path.joinpath(self.source_directory,"XR_SHOULDER")):
              print(study_category)
              print(type(study_category))
              study_str = str(study_category)
              tmp2 = Path.joinpath(self.source_directory,"XR_SHOULDER")
              print((tmp2))
              print(type(tmp2))
              str_tmp2 = str(tmp2)
              print(tmp2 == study_category)
              if tmp2 == study_category:
                print("paths printing hit")
                print(tmp2 == study_category)
              print("printing string")
              print(study_str)
              print(type(study_str))
              print(str_tmp2)
              print(type(str_tmp2))
              print(study_str == str_tmp2)
              if study_str == str_tmp2:
                print("hit here")
              

In [38]:
#Testing
objImg = Image_Loader("/content/drive/My Drive/AIML/Capstone/MURA-v1.1/","train",224,cv2.IMREAD_GRAYSCALE)

/content/drive/My Drive/AIML/Capstone/MURA-v1.1/train/XR_SHOULDER
<class 'pathlib.PosixPath'>
/content/drive/My Drive/AIML/Capstone/MURA-v1.1/train/XR_SHOULDER
<class 'pathlib.PosixPath'>
True
paths printing hit
True
printing string
/content/drive/My Drive/AIML/Capstone/MURA-v1.1/train/XR_SHOULDER
<class 'str'>
/content/drive/My Drive/AIML/Capstone/MURA-v1.1/train/XR_SHOULDER
<class 'str'>
True
hit here


In [0]:
class Image_Loader:
  
    def __init__(self, root_path, source_dir_name, size, img_read_format, img_type = ('*.jpg', '*.png')):
        self.project_path = root_path #root directory path of Mura-v1.1 dataset
        self.root = Path(self.project_path)
        self.source_directory = Path.joinpath(self.root, source_dir_name) #path from which image has to be read - Ex: train/test complete path
        self.scaled_size = size #Dimension to which image has to be rescaled
        self.formatRead = img_read_format #Format in which image has to be read : cv2.IMREAD_GRAYSCALE is by default used in this class by image_read_resize function#Future use #TODO
        self.img_format_type = img_type #image's actual format, currently .jpg, .png are default
        self.x_list=[]#empty list
        self.y_list=[]
        
    #Labelling Images
    #study1_positive and study1_negative are the only expected ones that can be processed as per MURA-v1.1 dataset
    def label_img(self,img):
      word_label = os.path.basename(os.path.dirname(img))
      print(word_label)
      if word_label == 'study1_positive': return 1
      elif word_label == 'study1_negative': return 0
      
      
      
    #Image_processing method
    def image_read_resize(self, file, scaled_size, formatRead = cv2.IMREAD_GRAYSCALE):
      #Reading image file in Grayscale mode, channel information removed
      #Datagenerator usage TODO
      formatRead = cv2.IMREAD_GRAYSCALE #set as default, also hardcoded as it's not possible to convert to numpy array without this#TODO
      OrigImage = cv2.imread(file,formatRead)
      #resizing image
      width = scaled_size
      height = scaled_size
      dim = (width, height)
      resizedImg = cv2.resize(OrigImage, dim)
      return resizedImg
    
    
    #Convert Image to list and numpy array
    def image_to_list_numpy(self):
      print(self.source_directory)
      for study_category in self.source_directory.iterdir():
        if(os.path.isdir(study_category)):#Only check for directories
          print(study_category)
          if study_category == (Path.joinpath(self.source_directory,"XR_SHOULDER")): #Only process shoulder, "currently ONLY hardcoded line", as it only needs shoulder, once this line is removed it will read all studies and all details
            patient_list = sum(1 for _ in study_category.iterdir()) # converting number of patients to list
            print(patient_list)
            for patient_id in study_category.iterdir():
              print(patient_id)
              for study_type in patient_id.iterdir():
                print(study_type)
                #img_format_type = ('*.jpg', '*.png') #TODO: Type of files this expects, need to add exceptions, atleast one general exception, log file,NotADirectoryError
                for files in self.img_format_type:
                  filelist = list(study_type.glob('**/'+ files))
                print(filelist)
                print(len(filelist))
                for fname in filelist:
                  class_label = self.label_img(fname) #Calling label_img function to get the actual label of the image
                  print(class_label)
                  for file in glob.glob(str(fname)):
                    print(file)
                    resizedImg = self.image_read_resize(file,self.scaled_size) #TODO: Read the image and resize it as per dimensions, maybe multiple uses of scaled_size param can be avoided as its part of class
                    self.x_list.append(resizedImg)
                    self.y_list.append(class_label)
                    x_npy = np.array(self.x_list)
                    y_npy = np.array(self.y_list)
      np.save((Path.joinpath(self.source_directory,"XR_SHOULDER","Images_array")), x_npy)#This indent has to match with  patient_list = sum(1 for _ in study_category.iterdir())*/ since only concerned abt shoulder
      np.save((Path.joinpath(self.source_directory,"XR_SHOULDER","Labels_array")), y_npy)
      return self.x_list, self.y_list, x_npy, y_npy

In [0]:
#Train dataset:
#Invoking / initializing class

objImg = Image_Loader("/content/drive/My Drive/AIML/Capstone/MURA-v1.1/","train",224,cv2.IMREAD_GRAYSCALE)

In [7]:
x_train, y_train, x_train_npy, y_train_npy  = objImg.image_to_list_numpy()

/content/drive/My Drive/AIML/Capstone/MURA-v1.1/train
/content/drive/My Drive/AIML/Capstone/MURA-v1.1/train/XR_HAND
/content/drive/My Drive/AIML/Capstone/MURA-v1.1/train/XR_SHOULDER
2
/content/drive/My Drive/AIML/Capstone/MURA-v1.1/train/XR_SHOULDER/patient02694
/content/drive/My Drive/AIML/Capstone/MURA-v1.1/train/XR_SHOULDER/patient02694/study1_negative
[PosixPath('/content/drive/My Drive/AIML/Capstone/MURA-v1.1/train/XR_SHOULDER/patient02694/study1_negative/image2.png'), PosixPath('/content/drive/My Drive/AIML/Capstone/MURA-v1.1/train/XR_SHOULDER/patient02694/study1_negative/image1.png')]
2
study1_negative
0
/content/drive/My Drive/AIML/Capstone/MURA-v1.1/train/XR_SHOULDER/patient02694/study1_negative/image2.png
study1_negative
0
/content/drive/My Drive/AIML/Capstone/MURA-v1.1/train/XR_SHOULDER/patient02694/study1_negative/image1.png
/content/drive/My Drive/AIML/Capstone/MURA-v1.1/train/XR_SHOULDER/patient02693
/content/drive/My Drive/AIML/Capstone/MURA-v1.1/train/XR_SHOULDER/patien

In [8]:
x_train_npy.shape, y_train_npy.shape #Number of images stored and its shape

((5, 224, 224), (5,))

In [0]:
Images = np.load('/content/drive/My Drive/AIML/Capstone/MURA-v1.1/train/XR_SHOULDER/Images_array.npy')
labels = np.load('/content/drive/My Drive/AIML/Capstone/MURA-v1.1/train/XR_SHOULDER/Labels_array.npy')

In [10]:
Images.shape, labels.shape

((5, 224, 224), (5,))